In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd
import urllib.request


In [ ]:
#df1 = pd.read_csv("small-2oq-c1r.csv", error_bad_lines=False, low_memory=False)
#print(df1)

#df2 = df1.set_index('categories', drop=False)
#df3 = df2.loc["Apparels>Women>Western Wear>Shirts, Tops & Tunics>Tops"]
#df3.to_csv('tops_listings_small.csv', index=False)

In [ ]:
df3 = pd.read_csv("tops_listings_small.csv")
df3.shape[0]


In [ ]:
from PIL import Image
import imagehash
import sys
import time

def similarImagesHash(products, n):
    similarImages = {}
    for i in range(0, n):
        urls = getImageUrls(products.iloc[i])
        #print("total images in this listing are :", len(urls))
        #urls[0]
    
        image = getImagesFromUrls(urls)
        #print(len(image))
        hashc = hashp = None
        hash0 = imagehash.average_hash(Image.fromarray(image[0]))
        
        #below for loop is to check if all images are similar for a given product
        for j in range (len(image)):
            #print(image[j].shape)
            hashc = imagehash.average_hash(Image.fromarray(image[j]))
            if (hashc != hashp):
                print("product ",i,": different hashc and hashp, images are not similar -", hashc, hashp)
            
            hashPrev = hashc

        #if hash0 in similarImages:
            #print(products.iloc[i].productId, '(for index', i,')  already exists as', ' '.join(similarImages[hash0]))
        
        if(i%10 == 0):
            print("Progress",i,"/",n, end="\r")
            #time.sleep(1)
            #sys.stdout.flush()
            
        similarImages[hash0] = similarImages.get(hash0, []) + [products.iloc[i].productId]

    return similarImages

In [ ]:
similarImages = similarImagesHash(df3, (int)(df3.shape[0]/1000.0))

In [ ]:
def getImageUrls(prod):
    url_str = prod.imageUrlStr
    img_urls = url_str.split(';')
    return img_urls

In [ ]:


def getImagesFromUrls(urls):
    img=[]
    for url in urls:
        req = urllib.request.urlopen(url)
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img.append(cv2.imdecode(arr, -1)) # 'Load it as it is'
        #print(url)
    return img


In [ ]:
# below is the test code
urls = getImageUrls(df3.iloc[0])

print("total images in this listing are :", len(urls))
urls[0]

image = getImagesFromUrls(urls)
print(len(image))
for i in range (len(image)):
    print(image[i].shape)


In [ ]:
#cv2.imshow('image', image[1])
#cv2.waitKey(2)

In [ ]:
#display one product with all the fields
df3.iloc[0]
#df3.iloc[53].description

In [ ]:
def areImagesSameForProducts(prod1, prod2):
    if(prod1.imageUrlStr == prod2.imageUrlStr):
        return True
    return False

In [ ]:
from scipy import spatial
from gensim import models

model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
index2word_set = set(model.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [ ]:
def similarText(text1, text2):
    s1_afv = avg_feature_vector(text1, model=model, num_features=300, index2word_set=index2word_set)
    s2_afv = avg_feature_vector(text2, model=model, num_features=300, index2word_set=index2word_set)
    sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
    #print(sim)
    return sim

simText = similarText(df3.iloc[10].title, df3.iloc[115].title)
simText = str(round(simText,4))
print(simText)

In [ ]:
#display the dictionary with all tuples with similar images (only primary image is taken into consideration)
for k, v in similarImages.items():
    print(k, v)

In [ ]:
def isDuplicate(prod1, prod2):
    if(prod1.productId == prod2.productId):
        return True
    if(prod1.productUrl == prod2.productUrl):
        return True
    
    if(prod1.title == prod2.title or prod1.description == prod2.description) :
        if(prod1.imageUrlStr != prod2.imageUrlStr):
            return True

    return False

In [ ]:
#df3.shape[0]
count = 0
for i in range(0, (int)(df3.shape[0]/1000.0)):
    for j in range(i+1, (int)(df3.shape[0]/1000.0)):
        if(isDuplicate(df3.iloc[i], df3.iloc[j])):
            count +=1
            print(count, i, j)
            #print("duplicate products:", i,j, df3.iloc[i].productUrl, df3.iloc[j].productUrl)

In [ ]:
a1 = 1.000234012
a1 = str(round(a1, 2))
print(a1)

a2 = 1.02524552
a2 = str(round(a2, 4))
print(a2)